In [1]:
import pandas as pd
import numpy as np
from cleanlab.filter import find_label_issues

pd.set_option("display.max_colwidth", None)

In [2]:
train_df = pd.read_csv("../data/cutoff_v2.csv")
prob_df = pd.read_csv("../data/probs.csv")

In [3]:
len(train_df), len(prob_df)

(42007, 42007)

In [4]:
prob_df["probs"] = prob_df["probs"].apply(lambda x: eval(x))

In [5]:
ordered_label_issues = find_label_issues(labels=prob_df['target'], # 데이터셋 라벨
                                         pred_probs=np.array(prob_df["probs"].to_list()), # 정답 예측 확률
                                         return_indices_ranked_by='self_confidence',)
len(ordered_label_issues)

3176

In [6]:
issue_df = prob_df.iloc[ordered_label_issues, :].reset_index(drop=True)

In [7]:
idx_to_text = ["IT과학", "경제", "사회", "생활문화", "세계", "스포츠", "정치"]

In [8]:
text_to_idx = {text: idx for idx, text in enumerate(idx_to_text)}

In [9]:
issue_df["pred"] = issue_df["probs"].apply(lambda x: np.argmax(x))

In [10]:
# 모델이 95% 이상 확률로 예측한 결과와 target이 다르다면 target이 잘못되었다고 판단
fatal_issue_df = issue_df[issue_df["probs"].apply(max) > 0.95].reset_index(drop=True)
len(fatal_issue_df)

831

In [11]:
fatal_issue_df.head()

,ID,target,probs,pred
0,ynat-v1_train_43587,3,"[0.0035275507252663374, 0.0043939026072621346, 0.010365847498178482, 0.002043513348326087, 0.013879763893783092, 0.003134066704660654, 0.9626554250717163]",6
1,ynat-v1_train_11475,3,"[0.0035611852072179317, 0.0038410453125834465, 0.009199091233313084, 0.0023092615883797407, 0.008370122872292995, 0.003002507146447897, 0.9697167873382568]",6
2,ynat-v1_train_06413,3,"[0.0028774600941687822, 0.004127013962715864, 0.012114248238503933, 0.0024750325828790665, 0.007154539227485657, 0.0030288679990917444, 0.9682229161262512]",6
3,ynat-v1_train_38468,3,"[0.0030039798002690077, 0.003943936433643103, 0.013137586414813995, 0.002609808463603258, 0.007909867912530899, 0.0026529175229370594, 0.9667419195175171]",6
4,ynat-v1_train_41801,3,"[0.003202886786311865, 0.0047707404009997845, 0.023398585617542267, 0.002672265050932765, 0.005640968680381775, 0.004050295799970627, 0.9562642574310303]",6


In [12]:
fatal_issue_df["text"] = train_df.set_index("ID").loc[fatal_issue_df["ID"]].reset_index()["text"]

In [13]:
fatal_issue_df["target_text"] = fatal_issue_df["target"].apply(lambda x: idx_to_text[x])
fatal_issue_df["pred_text"] = fatal_issue_df["pred"].apply(lambda x: idx_to_text[x])

In [14]:
# target_text에서 pred_text로 target을 수정
fatal_issue_df[["text", "target_text", "pred_text"]].sample(30)

,text,target_text,pred_text
777,MB 친형 압수수색에도 대응 없이 정중동…한국당도 신중종합,사회,정치
733,최고위원회의 발언하는 우원식,사회,정치
173,알사드 AFC챔피언스리그 4강 진출…정우영 풀타임,경제,스포츠
697,후 긴장 감도는 가자지구에 십싸만명부 늬료용품 공급,사회,세계
402,EU 융커 차기 집행위원장 후보 추천과정 투명하지 않았다,사회,세계
315,트럼프·엡스타인 1992년 마러라고서 여성 20여명과 파티,경제,세계
325,獨도끼만행범 허술한 신분검사 거쳐 헝가리서 입국,IT과학,세계
488,터키 EU 제재에도 동지중해 가스 시추 계속종합,정치,세계
799,與 비대위 전환도 교착…위원장·권한 놓고 힘겨루기,사회,정치
508,국내 주식형 펀드에 사흘 만에 자금 순유입,IT과학,경제


In [15]:
for idx, row in fatal_issue_df.iterrows():
    df_indexer = (train_df["ID"] == row["ID"])
    train_df.loc[df_indexer, "target"] = row["pred"]

In [16]:
train_df.drop_duplicates(subset=["text", "target"], inplace=True, ignore_index=True)

In [17]:
train_df.to_csv("../data/cutoff_v3.csv", index=False)